In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installing Packages

In [2]:
!pip install simpletransformers datasets tqdm pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6

In [19]:
import pandas as pd
from datasets import load_dataset
from tqdm import tqdm
from simpletransformers.t5 import T5Model
from sklearn.model_selection import train_test_split
import sklearn

In [4]:
dataset = load_dataset("HHousen/ParaSCI")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 338717
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 6433
    })
    test: Dataset({
        features: ['sentence1', 'sentence2'],
        num_rows: 4894
    })
})

In [9]:
def preprocess_dataset_ParaSCI(dataset):
  paraSCI = []
  for data in tqdm(dataset["train"]):
    keys = data.keys()
    paraSCI.append([data[key] for key in keys])

  paraSCI_data = pd.DataFrame(
      data = paraSCI,
      columns = [
          "sentence1",
          "sentence2"
      ]
  )

  paraSCI_data.to_csv("paraSCI_data.csv",sep="\t",index=None)
  return paraSCI_data

In [10]:
data = preprocess_dataset_ParaSCI(dataset)

100%|██████████| 338717/338717 [00:10<00:00, 33417.85it/s]


In [12]:
data.head()

,sentence1,sentence2
0,"for all methods , the tweets were tokenized wi...",the tweets were tokenized and part-ofspeech ta...
1,it was shown by nederhof et al that prefix pro...,"nederhof et al , for instance , show that pref..."
2,"first , kikuchi et al proposed a new long shor...","first , kikuchi et al tried to control the len..."
3,with word confusion networks further improves ...,the complexity is dominated by the word confus...
4,fofe can model the word order in a sequence ba...,fofe can model the word order in a sequence us...


In [13]:
!cp /content/paraSCI_data.csv /content/drive/MyDrive

In [14]:
data.tail()

,sentence1,sentence2
338712,al-halah and stiefelhagen construct the hierar...,"alternatively , al-halah and stiefelhagen expl..."
338713,deep neural networks have recently led to sign...,"in recent years , deep neural networks have be..."
338714,the formation of ultracold molecules has exper...,ultracold diatomic polar molecules have recent...
338715,for exchange and correlation the localdensity ...,the generalized gradient approximation of perd...
338716,we use adam optimizer to train the parameters .,we use the adam optimizer to update the parame...


In [17]:
data['sentence1'].iloc[338716]

'we use adam optimizer to train the parameters .'

In [18]:
data['sentence2'].iloc[338716]

'we use the adam optimizer to update the parameters .'

# Feature Eng

In [21]:
data["prefix"] = "paraphrase"
data.head()

,sentence1,sentence2,prefix
0,"for all methods , the tweets were tokenized wi...",the tweets were tokenized and part-ofspeech ta...,paraphrase
1,it was shown by nederhof et al that prefix pro...,"nederhof et al , for instance , show that pref...",paraphrase
2,"first , kikuchi et al proposed a new long shor...","first , kikuchi et al tried to control the len...",paraphrase
3,with word confusion networks further improves ...,the complexity is dominated by the word confus...,paraphrase
4,fofe can model the word order in a sequence ba...,fofe can model the word order in a sequence us...,paraphrase


In [25]:
data = data[:5000]
data.shape

(5000, 3)

In [52]:
train_data.iloc[39]['input_text']

'we use logistic regression as the per-class binary classifier , implemented using liblinear .'

In [33]:
data.columns = ["input_text","target_text","prefix"]
data.head()

,input_text,target_text,prefix
0,"for all methods , the tweets were tokenized wi...",the tweets were tokenized and part-ofspeech ta...,paraphrase
1,it was shown by nederhof et al that prefix pro...,"nederhof et al , for instance , show that pref...",paraphrase
2,"first , kikuchi et al proposed a new long shor...","first , kikuchi et al tried to control the len...",paraphrase
3,with word confusion networks further improves ...,the complexity is dominated by the word confus...,paraphrase
4,fofe can model the word order in a sequence ba...,fofe can model the word order in a sequence us...,paraphrase


# Model Training

In [34]:
train_data,test_data = train_test_split(data,test_size=0.1)

In [35]:
train_data.shape

(4500, 3)

In [36]:
test_data.shape

(500, 3)

In [37]:
args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "num_train_epochs": 4,
    "num_beams": None,
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "use_multiprocessing": False,
    "save_steps": -1,
    "save_eval_checkpoints": True,
    "evaluate_during_training": False,
    'adam_epsilon': 1e-08,
    'eval_batch_size': 6,
    'fp_16': False,
    'gradient_accumulation_steps': 16,
    'learning_rate': 0.0003,
    'max_grad_norm': 1.0,
    'n_gpu': 1,
    'seed': 42,
    'train_batch_size': 6,
    'warmup_steps': 0,
    'weight_decay': 0.0
}

In [38]:
model = T5Model("t5","t5-small", args=args)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [39]:
model.train_model(train_data, eval_data=test_data, use_cuda=True,acc=sklearn.metrics.accuracy_score)

  0%|          | 0/4500 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

Running Epoch 4 of 4:   0%|          | 0/750 [00:00<?, ?it/s]

(184, 2.2890862209479446)

#  Loading model to the drive

In [41]:
!zip /content/outputs_paraSCI_t5.zip /content/outputs

  adding: content/outputs/ (stored 0%)


In [42]:
!cp /content/outputs_paraSCI_t5.zip /content/drive/MyDrive/ParaSCI_T5

# Inference

In [43]:
from simpletransformers.t5 import T5Model
from pprint import pprint
import os

In [46]:
trained_model_path = '/content/outputs'

In [54]:
args = {
    "overwrite_output_dir": True,
    "max_seq_length": 256,
    "max_length": 50,
    "top_k": 50,
    "top_p": 0.95,
    "num_return_sequences": 1,
}

In [55]:
trained_model = T5Model("t5",trained_model_path,args=args)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [63]:
prefix = "paraphrase"
pred = trained_model.predict([f"{prefix}: we use logistic regression for regression problem"])
pprint(pred[0])

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/1 [00:00<?, ?it/s]

'we use logistic regression for regression problem analysis.'
